In [1]:
import pandas as pd
import numpy as np

# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [2]:
from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 100.0 # in meters
number_sites = 150
route = ApiRoute()
# route.get_route_data(current_position)
route.get_route_data(current_position, number_sites=number_sites)
# dataframe = route.get_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

      longitude   latitude  altitudeSmoothed  inclinationSmoothed  \
0    130.844161 -12.456694         16.350559            -0.024998   
1    131.001917 -12.478465         36.580360            -0.011472   
2    131.073619 -12.632972         36.058406             0.002710   
3    131.122372 -12.804406         38.282947             0.006234   
4    131.116697 -12.977285         86.538269            -0.007646   
..          ...        ...               ...                  ...   
145  138.275140 -34.331003         12.918265             0.005164   
146  138.387994 -34.484854         14.781113            -0.000336   
147  138.524375 -34.625967         15.796135            -0.000325   
148  138.579261 -34.791735          6.552269            -0.007854   
149  138.608989 -34.935232         56.847434            -0.016969   

      cumDistance  maxSpeed       theta  cumDistanceCut  
0    4.070956e+02        50   44.136411    0.000000e+00  
1    2.074550e+04        80  137.127468    2.033840e+04

,longitude,latitude,altitudeSmoothed,inclinationSmoothed,cumDistance,maxSpeed,theta,cumDistanceCut
0,130.844161,-12.456694,16.350559,-0.024998,4.070956e+02,50,44.136411,0.000000e+00
1,131.001917,-12.478465,36.580360,-0.011472,2.074550e+04,80,137.127468,2.033840e+04
2,131.073619,-12.632972,36.058406,0.002710,4.108390e+04,100,170.546248,4.067680e+04
3,131.122372,-12.804406,38.282947,0.006234,6.142230e+04,130,183.305486,6.101520e+04
4,131.116697,-12.977285,86.538269,-0.007646,8.176070e+04,130,176.253243,8.135360e+04
...,...,...,...,...,...,...,...,...
145,138.275140,-34.331003,12.918265,0.005164,2.949475e+06,110,122.443515,2.949068e+06
146,138.387994,-34.484854,14.781113,-0.000336,2.969814e+06,110,142.165207,2.969407e+06
147,138.524375,-34.625967,15.796135,-0.000325,2.990152e+06,110,156.484508,2.989745e+06
148,138.579261,-34.791735,6.552269,-0.007854,3.010490e+06,110,193.271020,3.010083e+06


In [3]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
      name site_id  longitude  latitude altitude
0  lerici  584990    9.88774   44.1824     1300


In [ ]:
# api_executer.add_sites(dataframe, print_is_requested=False)
api_executer.delete_all_sites()

In [ ]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(584978, True, name="aa")
api_requester.get_site_edit(584979, True, position=position)
api_requester.get_site_edit(584980, True, name="df", position=position)

In [4]:
forecasts = api_requester.get_solar_forecast()
# forecasts = api_requester.get_solar_forecast_cloudmove()
print(forecasts)

Response status from getforecast: OK.
Solar forecast have been retrieved.
                                gk      gh_max   gh         tt         ff  \
site_id time                                                                
584990  2023-08-30 08:00:00  281.0  590.478901  281  11.400000   8.000000   
        2023-08-30 08:10:00  309.0  620.069133  309  11.550000   7.666667   
        2023-08-30 08:20:00  338.0  648.611457  338  11.700000   7.333333   
        2023-08-30 08:30:00  367.0  676.036070  367  11.850000   7.000000   
        2023-08-30 08:40:00  397.0  702.278323  397  12.000000   6.666667   
...                            ...         ...  ...        ...        ...   
        2023-09-08 06:20:00   53.0  225.083883   53  13.933333  11.000000   
        2023-09-08 06:30:00   73.0  259.389989   73  14.000000  11.000000   
        2023-09-08 06:40:00   98.0  293.896812   98  14.066667  11.000000   
        2023-09-08 06:50:00  126.0  328.398057  126  14.133333  11.000000   
  

In [7]:
index = forecasts.index

times = pd.to_datetime(index.get_level_values('time')).hour
print(times)

# Access the values of time and site_id
time_values = index.get_level_values('time')
site_id_values = index.get_level_values('site_id')

# Print the values
print("Time values:", time_values)
print("Site ID values:", site_id_values)

Index([8, 8, 8, 8, 8, 8, 9, 9, 9, 9,
       ...
       5, 5, 5, 6, 6, 6, 6, 6, 6, 7],
      dtype='int32', name='time', length=1291)
Time values: Index(['2023-08-30 08:00:00', '2023-08-30 08:10:00', '2023-08-30 08:20:00',
       '2023-08-30 08:30:00', '2023-08-30 08:40:00', '2023-08-30 08:50:00',
       '2023-08-30 09:00:00', '2023-08-30 09:10:00', '2023-08-30 09:20:00',
       '2023-08-30 09:30:00',
       ...
       '2023-09-08 05:30:00', '2023-09-08 05:40:00', '2023-09-08 05:50:00',
       '2023-09-08 06:00:00', '2023-09-08 06:10:00', '2023-09-08 06:20:00',
       '2023-09-08 06:30:00', '2023-09-08 06:40:00', '2023-09-08 06:50:00',
       '2023-09-08 07:00:00'],
      dtype='object', name='time', length=1291)
Site ID values: Index(['584990', '584990', '584990', '584990', '584990', '584990', '584990',
       '584990', '584990', '584990',
       ...
       '584990', '584990', '584990', '584990', '584990', '584990', '584990',
       '584990', '584990', '584990'],
      dtype='object', 

In [ ]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('rh')]
rr_values = rr_values.to_frame()
print(rr_values)
print(rr_values.max(), rr_values.min())
# # Assuming your MultiIndex DataFrame is named "forecast_data"
# pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
# print(pivot_table)
rr_values.to_csv('rh_table.csv')

# print(rr_values)